In [1]:
import pandas as pd
import numpy as np

Пусть имеются данные о займах (loans), выданных различными организациями (originators). Уникальный идентификатор займа - loanId, уникальный идентификатор организации - originatorId. Из параметров займа известны сумма - amount, срок - term, комиссии - fees, дата выдачи в формате UNIXTIMESTAMP - issuedDatetime, дата погашения в формате UNIXTIMESTAMP - repaidDatetime. Организации находятся в разных странах, и сумма займа отображается в валюте конкретной организации. Таблица originators содержит соотношения между организациями originators и валютами currency. Для каждой организации может быть только одна валюта.

In [2]:
loans = pd.read_csv('data/loans.csv')
loans.head()

,originatorId,loanId,amount,term,rate,fees,issuedDatetime,repaidDatetime
0,2,1492427,200.0,20,NaN,61.0,1438276988,1.440170e+09
1,2,1360769,700.0,30,NaN,0.0,1432733814,1.440168e+09
2,2,1453132,800.0,30,NaN,0.0,1437469946,1.440146e+09
3,2,1514642,500.0,10,NaN,99.0,1439402251,1.440141e+09
4,2,1537517,500.0,30,NaN,0.0,1440499423,1.441029e+09


In [3]:
loans['issuedDatetime'] = pd.to_datetime(loans['issuedDatetime'], unit = 's')

In [4]:
originators = pd.read_csv('data/originators.csv')
originators.head()

,originatorId,currency
0,1,1
1,2,7
2,6,3
3,8,7
4,16,8


Также имеются исторические данные о курсах валют в таблице rates. Представлено значение курса одной валюты (sourceCurrencyId) к другой валюте (targetCurrencyId) в конкретный день day. Предполагается, что в течение дня курс валют не изменялся.

In [5]:
rates = pd.read_csv('data/currency_rates.csv')
rates.head()

,day,sourceCurrencyId,targetCurrencyId,rate
0,20151008,1,5,0.020877
1,20151008,1,4,0.010473
2,20151008,1,7,0.060175
3,20151008,1,2,0.016002
4,20151008,1,3,0.014219


In [6]:
rates['day'] = pd.to_datetime(rates['day'], format='%Y%m%d', errors='ignore')

### На какую сумму, приведенную к валюте 4, каждая из огранизаций выдала займов в июне 2017 года?

In [7]:
def Transf(val1, val2 = 4, time ='2017-06-30'):
    if val1 in rates['sourceCurrencyId'].values and val2 in rates['targetCurrencyId'].values and val1!=val2:
        rez = rates[(rates['sourceCurrencyId'] == val1) & 
                    (rates['targetCurrencyId'] == val2) &
                    (rates['day'] <= time)].sort_values(by = ['day'], ascending=False)['rate'].head(1).as_matrix()[0]
        return rez
    elif val1==val2:
        return 1
    else:
        return np.nan

In [8]:
loans['amount4'] = loans[(loans['issuedDatetime'] >= '2017-06') & ((loans['issuedDatetime'] < '2017-07'))][['originatorId', 'amount', 'issuedDatetime']].apply(lambda row: Transf(originators[originators['originatorId']==row.as_matrix()[0]]['currency'].as_matrix()[0], time = row.as_matrix()[2])*row.as_matrix()[1], axis=1)

In [9]:
loans[(loans['amount4'].isnull()==False) & (loans['amount4'].isnull()==False)]

,originatorId,loanId,amount,term,rate,fees,issuedDatetime,repaidDatetime,amount4
62675,2,3799148,3205.49,30,NaN,664.00,2017-06-01 09:55:56,NaN,668.440830
62679,2,3441064,4487.69,30,NaN,0.00,2017-06-01 09:55:34,NaN,935.817996
62682,2,3381924,6410.23,30,NaN,0.00,2017-06-01 11:21:47,NaN,1336.725262
62687,2,3797825,2500.00,30,NaN,518.00,2017-06-01 13:47:02,1.498774e+09,521.325000
62689,2,3441083,1100.00,25,NaN,0.00,2017-06-01 07:16:44,NaN,229.383000
62698,2,3800082,5513.45,30,NaN,1142.00,2017-06-01 13:45:51,1.499433e+09,1149.719728
62707,2,3792308,4487.69,30,NaN,0.00,2017-06-01 12:22:24,NaN,935.817996
62724,8,3803733,926.39,20,NaN,0.00,2017-06-01 15:57:34,NaN,193.180107
62731,8,3706814,5658.83,10,NaN,0.00,2017-06-01 12:17:48,NaN,1180.035820
62738,8,3639485,1000.00,30,NaN,168.30,2017-06-01 13:06:40,1.498828e+09,208.530000
